# Part 3 - Static Replication

Static Replication of exotic European derivative contract using Black-Schloes, Bachelier and SABR with a following payoff:

$$
S^{1/3}_T + 1.5 \cdot \log(S_T) + 10.0
$$

In [63]:
%autosave 60
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.stats import norm
from scipy.optimize import brentq
import matplotlib.pylab as plt

Autosaving every 60 seconds


In [64]:
import warnings
warnings.filterwarnings('ignore')

In [65]:
#S&P500 index data
spx = pd.read_csv('Data/SPX_options.csv',
                  index_col=0,
                  parse_dates=True)

#Discount rates
discount_rate = pd.read_csv('Data/zero_rates_20201201.csv',
                           index_col=0,
                           parse_dates=True)

In [66]:
def preprocessData(df):
    df['mid'] = 0.5*(df['best_bid'] + df['best_offer'])
    df['strike'] = df['strike_price']*0.001
    df['payoff'] = df['cp_flag'].map(lambda x: 'call' if x == 'C' else 'put')
    return df

In [67]:
spx = preprocessData(spx)

Lets calculate the implied volatility based on the market data.

In [68]:
def impliedVolatility(S, K, r, price, T, payoff):
    try:
        if (payoff.lower() == 'call'):
            impliedVol = brentq(lambda x: price -
                                BlackScholesLognormalCall(S, K, r, x, T),
                                1e-12, 10.0)
        elif (payoff.lower() == 'put'):
            impliedVol = brentq(lambda x: price -
                                BlackScholesLognormalPut(S, K, r, x, T),
                                1e-12, 10.0)
        else:
            raise NameError('Payoff type not recognized')
    except Exception:
        impliedVol = np.nan

    return impliedVol


def BlackScholesLognormalCall(S, K, r, sigma, T):
    d1 = (np.log(S/K)+(r+sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)


def BlackScholesLognormalPut(S, K, r, sigma, T):
    d1 = (np.log(S/K)+(r+sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return K*np.exp(-r*T)*norm.cdf(-d2) - S*norm.cdf(-d1)

In [69]:
spx.exdate.value_counts()

exdate
20201218    822
20210115    738
20210219    512
Name: count, dtype: int64

In [70]:
data = \
    spx[spx.exdate == 20210115]

In [82]:
# Lets calculate the implied volatility for the ATM option
K = 3660
atm = data[data.strike == K]

In [86]:
atm_call = \
    atm[atm['payoff'] == 'call']

atm_put = \
    atm[atm['payoff'] == 'put']

In [112]:
#Define parameters
S = 3662.45
days_to_expiry = (pd.Timestamp(str(20210115)) - pd.Timestamp('2020-12-01')).days
T = days_to_expiry/365

#calculate the discount rate
r = np.interp(days_to_expiry, discount_rate['days'], discount_rate['rate'])

In [88]:
#calculate implied volatilities from market data
atm_call['vols'] = atm_call.apply(lambda x: impliedVolatility(S,
                                                  x['strike'],
                                                  r,
                                                  x['mid'],
                                                  T,
                                                  x['payoff']),
                                                  axis=1)

In [89]:
atm_put['vols'] = atm_put.apply(lambda x: impliedVolatility(S,
                                                  x['strike'],
                                                  r,
                                                  x['mid'],
                                                  T,
                                                  x['payoff']),
                                                  axis=1)

In [102]:
atm_sigma = (atm_call['vols'].values[0] + atm_put['vols'].values[0])/2

In [100]:
# Lets calculate the price of Exotic derivative using black scholes

def black_scholes_pricing(r,T,S,sigma):
    return np.exp(-r*T)*((S**(1/3)*np.exp((r-(sigma**2/6))*(T/3)) + 1.5*np.log(S) + 1.5*(r-(sigma**2/2))*T + 10))

In [103]:
black_scholes_pricing(r,T,S,atm_sigma)

36.94057587959251

In [105]:
# Lets calculate the price of Exotic derivative using bachelier model

def bachelier_pricing(r,T,S,sigma):
    return np.exp(-r*T)*(S**(1/3) + 1.5*np.log(S) - ((3*sigma**2*T)/(4*S**2)) + 10)

In [106]:
bachelier_pricing(r,T,S,atm_sigma)

36.78118095136091

For any twice-differentiable payoff $h(S_T)$, Breeden-Litzenberger states that

\begin{equation*}
    \begin{split}
      V_0 = e^{-rT}h(F) + \underbrace{\int_0^{F}h''(K)P(K)\;dK}_{\mbox{put integral}} + \underbrace{\int_{F}^{\infty}h''(K)C(K)\;dK}_{\mbox{call integral}}\\
    \end{split}
\end{equation*}

In [114]:
# Lets calculate the implied volatility using SABR model
def SABR(F, K, T, alpha, beta, rho, nu):
    X = K
    # if K is at-the-money-forward
    if abs(F - K) < 1e-12:
        numer1 = (((1 - beta)**2)/24)*alpha*alpha/(F**(2 - 2*beta))
        numer2 = 0.25*rho*beta*nu*alpha/(F**(1 - beta))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        VolAtm = alpha*(1 + (numer1 + numer2 + numer3)*T)/(F**(1-beta))
        sabrsigma = VolAtm
    else:
        z = (nu/alpha)*((F*X)**(0.5*(1-beta)))*np.log(F/X)
        zhi = np.log((((1 - 2*rho*z + z*z)**0.5) + z - rho)/(1 - rho))
        numer1 = (((1 - beta)**2)/24)*((alpha*alpha)/((F*X)**(1 - beta)))
        numer2 = 0.25*rho*beta*nu*alpha/((F*X)**((1 - beta)/2))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        numer = alpha*(1 + (numer1 + numer2 + numer3)*T)*z
        denom1 = ((1 - beta)**2/24)*(np.log(F/X))**2
        denom2 = (((1 - beta)**4)/1920)*((np.log(F/X))**4)
        denom = ((F*X)**((1 - beta)/2))*(1 + denom1 + denom2)*zhi
        sabrsigma = numer/denom

    return sabrsigma

In [122]:
#lets use the calibrated model parameters 
alpha = 1.725
beta = 0.7
rho = -0.667
nu = 2.862
F = S * np.exp(r*T)

In [123]:
# price the option using SABR implied Volatility
def SABRCall(S, K, r, alpha, beta, rho, nu, T):
    sabr_vol = SABR(S*np.exp(r*T), K, T, alpha, beta, rho, nu)
    return BlackScholesCall(S, K, r, sabr_vol, T)


def SABRPut(S, K, r, alpha, beta, rho, nu, T):
    sabr_vol = SABR(S*np.exp(r*T), K, T, alpha, beta, rho, nu)
    return BlackScholesPut(S, K, r, sabr_vol, T)

In [125]:
def call_integrand(K, S, r, T, alpha, beta, rho, nu):
    hK = (-2/(9*K**(5/3))) - (3/(2*K**2))
    price = SABRCall(S, K, r, alpha, beta, rho, nu, T)
    return hK*price

def put_integrand(K, S, r, T, alpha, beta, rho, nu):
    hK = (-2/(9*K**(5/3))) - (3/(2*K**2))
    price = SABRPut(S, K, r, alpha, beta, rho, nu, T)
    return hK*price

In [135]:
#lets calculate the price of the exotic derivative using SABR volatility

def sabr_pricing(r, T, F, S, alpha, beta, rho, nu):
    first_term = np.exp(-r*T)*(F**(1/3) + 1.5*np.log(F) + 10)
    I_put = quad(lambda x: put_integrand(x, S, r, T, alpha, beta, rho, nu), 1e-6, F)
    I_call = quad(lambda x: call_integrand(x, S, r, T, alpha, beta, rho, nu), F, 7000)
    price = first_term + I_put[0] + I_call[0]
    return price

In [136]:
sabr_pricing(r, T, F, S, alpha, beta, rho, nu)

36.931949069031965

> Price using Black Scholes : 36.94057587959251


> Price using Bachelier : 36.78118095136091


> Price using SABR : 36.931949069031965